 # On a assez d’ingrédients, c’est le moment de créer une appli RAG simple

In [5]:
import os
import gradio as gr
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings

# Étape 1 : Lire le document Markdown
def load_docs():
    docs = []
    for root, _, files in os.walk("docs"):
        for filename in files:
            if filename.endswith(".md"):
                full_path = os.path.join(root, filename)
                loader = TextLoader(full_path, encoding='utf-8')
                docs.extend(loader.load())
    return docs

# Étape 2 : Utiliser ChromaDB pour créer ou charger une base de données vectorielle

# Modèle d'embedding : MiniLM exécuté en local
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

embeddings = OllamaEmbeddings(model="nomic-embed-text")

def get_vectorstore():
    persist_dir = "chroma_store"
    collection_meta = {"hnsw:space": "cosine"}
    if os.path.exists(persist_dir):
        # Charger une base de données existante
        vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embeddings)
    else:
        # Créer une nouvelle base de données.
        documents = load_docs()
        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        texts = splitter.split_documents(documents)
        #vectorstore = Chroma.from_documents(texts, embedding=embeddings, persist_directory=persist_dir)
        vectorstore = Chroma.from_documents(
                    texts,
                    embedding=embeddings,
                    persist_directory=persist_dir,
                    collection_metadata=collection_meta
        )
        vectorstore.persist()  # Enregistrer dans la base
    return vectorstore

# Modèle LLM local : utilisation de Ollama pour appeler Mistral
llm = OllamaLLM(model="mistral", temperature=0.7)

# Créer un système de récupération RAG
vectorstore = get_vectorstore()
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

def chat_with_rag(user_input):
    
# result est un dictionnaire au format suivant :
# {
#  "result": "Texte final de la réponse générée par le modèle",
#  "source_documents": [document1, document2, document3]
# }

    result = qa_chain.invoke(user_input)
    answer = result["result"]
    sources = result.get("source_documents", [])
    file_list = set()
    for doc in sources:
        if hasattr(doc.metadata, "get"):
            source = doc.metadata.get("source")
            if source:
                file_list.add(source) 
    if file_list:
        answer += "\n\n📄 Le fichier:\n"
        for file in sorted(file_list):
            url = path_to_gitlab_url(file)
            norm_path = file.replace("\\", "/")
            answer += f"- `{norm_path}`\n  🔗({url})\n"
    return answer
    
# Interface de chat Gradio
view = gr.Interface(
    fn=chat_with_rag,
    inputs=gr.Textbox(label="Your message:", lines=6, placeholder="Veuillez entrer votre question..."),
    outputs=gr.Textbox(label="Response:", lines=8),
    title="RAG Assistant (Ollama + ChromaDB)",
    flagging_mode="never"
)

def path_to_gitlab_url(path):
    # Remplacer les séparateurs de chemin Windows par le format URL
    url_path = path.replace("\\", "/")
    # Supprimer le préfixe
    if url_path.startswith("docs/wiki-developpement-squash.wiki/"):
        url_path = url_path[len("docs/wiki-developpement-squash.wiki/"):]
    # Supprimer le suffixe .md (il n’y a pas de .md dans l’URL du wiki GitLab)
    if url_path.endswith(".md"):
        url_path = url_path[:-3]
    return f"https://gitlab.com/henixdevelopment/squash/wiki-developpement-squash/-/wikis/{url_path}"

view.launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://f307200ecd327d11fb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
